In [1]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import requests
# from bs4 import BeautifulSoup
# from unidecode import unidecode
# from collections import Counter
# from datetime import datetime, timedelta
from team_maker import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [69]:
# by_name = df.set_index('Name')
tnames = ['Blake Snell', 'Mike Foltynewicz']
by_name.loc[tnames]

,Position,Name + ID,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
Name,,,,,,,,
Blake Snell,SP,Blake Snell (11121148),11121148,P,11500,KC@TB 08/23/2018 07:10PM ET,TB,22.67
Mike Foltynewicz,SP,Mike Foltynewicz (11121149),11121149,P,11100,ATL@MIA 08/23/2018 07:10PM ET,ATL,20.75


In [17]:
positions = ['C', '1B', '2B', 'SS', '3B', 'OF', 'OF', 'OF', 'P', 'P']

# df = pd.read_csv('DKSalaries.csv')

# df = prep_dataframe()
# df
# df['PPK'] = df.AvgPointsPerGame / df.Salary


# eff = pd.concat(baselist)
# eff.AvgPointsPerGame.sum()
# eff.sort_values('AvgPointsPerGame')
# pd.concat(,  ].extend(baselist))

# randomized_best_team(df, positions)

# np.random.choice()
# make_n_teams(1, positions)
# choose_top_players_pos(df, positions)
df = prep_dataframe()
df
# get_projected_players()

Extracting csv
Filtering by projected players
Expanding dataframe
Getting pitcher and team score information


,AvgPointsPerGame,Game Info,ID,MULT,NAME,Name,Name + ID,OppPitcher,OpposingTeam,PPK,Position,Roster Position,Salary,Team,TeamAbbrev,index,Adjusted
0,7.88,TEX@LAA 09/10/2018 10:07PM ET,11236317,NaN,NaN,Jurickson Profar,Jurickson Profar (11236317),NaN,LAA,1.751111,3B/SS,3B/SS,4500,NaN,TEX,NaN,NaN
1,8.25,CLE@TB 09/10/2018 07:10PM ET,11236335,NaN,NaN,Edwin Encarnacion,Edwin Encarnacion (11236335),NaN,TB,2.062500,1B,1B,4000,NaN,CLE,NaN,NaN
2,4.83,WAS@PHI 09/10/2018 07:05PM ET,11236412,NaN,NaN,J.P. Crawford,J.P. Crawford (11236412),NaN,WAS,1.238462,3B/SS,3B/SS,3900,NaN,PHI,NaN,NaN
3,5.46,Postponed,11236416,NaN,NaN,Jay Bruce,Jay Bruce (11236416),NaN,Postponed,1.436842,OF,OF,3800,NaN,NYM,NaN,NaN
4,5.19,NYY@MIN 09/10/2018 08:10PM ET,11236559,NaN,NaN,Robbie Grossman,Robbie Grossman (11236559),NaN,NYY,1.621875,OF,OF,3200,NaN,MIN,NaN,NaN
5,5.30,CWS@KC 09/10/2018 08:15PM ET,11236627,NaN,NaN,Omar Narvaez,Omar Narvaez (11236627),NaN,KC,1.766667,C,C,3000,NaN,CWS,NaN,NaN


In [26]:
eff.AvgPointsPerGame.sum()

105.0

In [110]:
def get_pitcher_adjustments():
    """
    scrapes data from baseball prospectus and returns a dataframe with a 
    multiplyer for each pitcher based on their WHIP
    """
    rq = requests.get('https://legacy.baseballprospectus.com/sortable/index.php?cid=2508773').content
    souped = BeautifulSoup(rq, 'html.parser')
    ttdata = souped.find('table', {'id' : 'TTdata'})
    hdrs = [x.text for x in ttdata.find('tr').findAll('td')]
    data = []
    for r in ttdata.findAll('tr')[1:]:
        data.append([x.text for x in r.findAll('td')])

    pf = pd.DataFrame(data, columns = hdrs)
    numheaders = list(pf)[3:]
    for header in numheaders:
        pf[header] = pd.to_numeric(pf[header])
    pf = pf[pf['G'] > (pf['G'].mean() - pf["G"].std())]
    pf['WHIP'] = (pf['H'] + pf['BB'] + pf['HBP']) / pf['IP']
    # pf.dtypes
    pf['MULT'] = pf['WHIP'] / pf['WHIP'].mean()
    return pf[['NAME', 'MULT']]    

def get_projected_players():
    """
    Gets projected players from 
    https://rotogrinders.com/lineups/mlb?site=draftkings
    """
    players = []
    
    rq = requests.get('https://rotogrinders.com/lineups/mlb?site=draftkings').content

    souped = BeautifulSoup(rq, 'html.parser')

    divs = souped.findAll('div', {'class' : 'blk game'})
    for div in divs:
        
        hm = div.find('div', {'class' : 'blk home-team'})
        aw = div.find('div', {'class' : 'blk away-team'})

        hp = hm.find('div', {'class' : 'pitcher players'}).find('a', {'class' : 'player-popup'}).text
        ap = aw.find('div', {'class' : 'pitcher players'}).find('a', {'class' : 'player-popup'}).text

        ht = hm.find('ul', {'class' : 'players unconfirmed'})#.findAll('li', {'class' : 'player'})
        at = aw.find('ul', {'class' : 'players unconfirmed'})#.findAll('li', {'class' : 'player'})

        ht = ht.findAll('li', {'class' : 'player'})
        ht = [x.find('span', {'class' : 'pname'}).text.strip() for x in ht]

        at = at.findAll('li', {'class' : 'player'})
        at = [x.find('span', {'class' : 'pname'}).text.strip() for x in at]

        players.extend([hp, ap])
        players.extend(at)
        players.extend(ht)

    return players

def choose_top_players_pos(df, positions):
    """
    df = DataFrame of all available players
    positions = list of strings, positions to be staffed,
                each position should be in the list the number
                of times it is in the roster
                
    returns a dataframe of the top n players for each position, 
    where n is the number of times the position appears in the list
    """
    df = df.set_index(['Roster Position'])
    
    # Getting counter dict of positions
    pos_count = Counter(positions)
    
    # Creating a list of mini dfs with top players by position
    baselist = [df.loc[x][:pos_count[x]] for x in pos_count]
   
    # Making one big dataframe and concatenating
    return pd.concat(baselist)

def choose_top_players_flex(df, num):
    """
    df = DataFrame of all available players
    num = int, number of players to choose
    
    returns a dataframe of the top n players, where n == num
    """
    
    # Removing injured players from the dataframe
    inj = get_injured_players('baseball')
    df = df[~df['Name'].isin(inj)]
    
    # Returning top players
    return df[:num]

def get_injured_players(sport):
    """
    sport = str, the targeted sport
    """
    
    # Dict of pages for injury reports
    urls = {
        'baseball' : 'https://scores.nbcsports.com/mlb/stats.asp?file=inj'
    }
    url = urls[sport]
    
    # Getting html of injury report page
    rq = requests.get(url)
    souped = BeautifulSoup(rq.content, 'html.parser')
    tds = souped.find_all('td')
    
    # Creating list of injured players
    inj = []
    for n in range(5, len(tds), 3):
        
        td = tds[n].getText()
        if ',' in td:
            
            # Matching source unicode formatting
            td = unidecode(td.split(',')[0])
            
            inj.append(td)

    return inj
    

def make_efficient_team(df, positions = None, num = None):
    """
    df = DataFrame of all available players
    positions = list of strings
    num = int
    
    Returns a dataframe of a team with the best ratio of 
    average points to cost.
    """
    df = df.sort_values('PPK', ascending=False)
    
    if positions != None: 
        return choose_top_players_pos(df, positions)
    if num != None:
        return choose_top_players_flex(df, num)
        

def make_best_team(df, positions = None, num = None):
    """
    df = DataFrame of all available players
    positions = list of strings
    num = int
    
    Returns a dataframe of a team with the best average points.
    """

    df = df.sort_values('AvgPointsPerGame', ascending=False)
    
    if positions != None: 
        return choose_top_players_pos(df, positions)
    if num != None:
        return choose_top_players_flex(df, num)
    
def make_team_w_pos(positions):
    """
    positions = list of strings, positions to be staffed,
                each position should be in the list the number
                of times it is in the roster
    
    returns a dataframe of a team formed with the efficient to
    best method. Starts with the most efficient players for 
    their price, then adds the best players, regardless of price, 
    while keeping the whole team's price under the cap
    """
   
    df = prep_dataframe()
    
    eff = make_efficient_team(df, positions = positions)
    best = make_best_team(df, positions = positions)
    
    # 'DEFICIT' = The expectation of best - efficient player
    eff['DEFICIT'] = best.AvgPointsPerGame - eff.AvgPointsPerGame
    best['DEFICIT'] = best.AvgPointsPerGame - eff.AvgPointsPerGame
    
    eff = eff.sort_values('DEFICIT', ascending=False).reset_index() 
    best = best.sort_values('DEFICIT', ascending=False).reset_index()
    
    # Replacing efficient players with best players in deficit order
    for i, e in eff.iterrows():

        ctotal = eff.Salary.sum()
        ctminus = ctotal - e.Salary

        if ctminus + best.Salary.loc[i] <= 50000:
            if best.Name.loc[i] not in eff.Name.unique():
                eff.loc[i] = best.loc[i]
            
    print('Salary:', eff.Salary.sum())
    print('Avg Points:', eff.AvgPointsPerGame.sum())
    print('Avg w/o cap:', best.AvgPointsPerGame.sum())
    print('Sal w/o cap:', best.Salary.sum())
    
    return eff

def make_team_flex(num):
    """
    num = int, number of players to include on the final roster
    
    returns a dataframe of a team formed with the efficient to
    best method. Starts with the most efficient players for 
    their price, then adds the best players, regardless of price, 
    while keeping the whole team's price under the cap
    """
    
    df = prep_dataframe()
    
    # num quadrupled for 'best' to give more options
    eff = make_efficient_team(df, num =num).reset_index()
    best = make_best_team(df, num = num*4).reset_index()
    
    # 'DEFICIT' = The expectation of best - efficient player
    eff['DEFICIT'] = best.AvgPointsPerGame - eff.AvgPointsPerGame
    best['DEFICIT'] = best.AvgPointsPerGame - eff.AvgPointsPerGame
    
    eff = eff.sort_values('DEFICIT', ascending=False).reset_index() 
    best = best.sort_values('AvgPointsPerGame', ascending=False).reset_index()
    
    # Replacing efficient players one at a time while staying under cap
    for i, e in eff.iterrows():
        
        for j, r in best.iterrows():
            
            ctotal = eff.Salary.sum()
            ctminus = ctotal - e.Salary

            if ctminus + best.Salary.loc[j] <= 50000 \
            and best.Name.loc[j] not in eff.Name.unique()\
            and e.AvgPointsPerGame < r.AvgPointsPerGame:
                
                eff.loc[i] = best.loc[j]
                break
            
    print('Salary:', eff.Salary.sum())
    print('Avg Points:', eff.AvgPointsPerGame.sum())
    print('Avg w/o cap:', best.AvgPointsPerGame[:num].sum())
    print('Sal w/o cap:', best.Salary[:num].sum())
    
    return eff, best

def prep_dataframe():
    """
    Builds a dataframe from DKSalaries.csv and prepares it
    for use by other functions
    """
    df = pd.read_csv('DKSalaries.csv')
    
    # Adding opposing team info
    all_matchups = []
    for x in df['Game Info']:
        ats = x.split('@')
        t1 = ats[0]
        t2 = ats[1].split(' ')[0]
        all_matchups.append([t1, t2])

    OpposingTeam = []
    for mu, ta in zip(all_matchups, df.TeamAbbrev):
        if mu[0] == ta:
            OpposingTeam.append(mu[1])
        else:
            OpposingTeam.append(mu[0])

    df['OpposingTeam'] = OpposingTeam


    df['PPK'] = df.AvgPointsPerGame / df.Salary * 1000
#     inj = get_injured_players('baseball')
#     df = df[~df['Name'].isin(inj)]
#     df = clear_dead_weight(df.set_index('Roster Position'))
    act = get_projected_players()
    df = df[df['Name'].isin(act)]
    
    return df#.reset_index()

def clear_dead_weight(df):
    
    df = df.sort_values('PPK', ascending=False)
    output = pd.DataFrame()
    
    for pos in [x for x in df.index.unique() if len(x) < 3]:
        print(pos)
        tdf = df.loc[pos]
#         return tdf
        floor = tdf.AvgPointsPerGame.iloc[0]
        tdf = tdf[tdf['AvgPointsPerGame'] >= floor]
#         tdf['Roster Position'] = pos

        output = pd.concat([output, tdf])
    
#     output = output.set_index('Roster Position')
    return output

def create_active_player_filter():
    """
    Gets lineups from news sites and returns a list of players who are
    playing tonight.
    """
    rq = requests.get('https://www.mlb.com/starting-lineups').content
    souped = BeautifulSoup(rq, 'html.parser')

    divs = souped.findAll("div", {"class": "starting-lineups__matchup"})

    avails = pd.DataFrame()
    all_sps = []

    for div in divs:
        htm = div.find("span", {"class": "starting-lineups__team-name starting-lineups__team-name--home"}).text.strip()
        atm = div.find("span", {"class": "starting-lineups__team-name starting-lineups__team-name--away"}).text.strip()

        htmlu = div.find("ol", {"class": "starting-lineups__team starting-lineups__team--home"})
        atmlu = div.find("ol", {"class": "starting-lineups__team starting-lineups__team--away"})

        htmbs = htmlu.findAll("li", {"class": "starting-lineups__player"})
        atmbs = atmlu.findAll("li", {"class": "starting-lineups__player"})

        sps = div.findAll("div", {"class" : "starting-lineups__pitcher-name"})
        all_sps.extend([x.text.strip() for x in sps])

        htmbs = pd.DataFrame([x.text.split(' ') for x in htmbs])
        htmbs[4] = htm
        atmbs = pd.DataFrame([x.text.split(' ') for x in atmbs])
        atmbs[4] = atm

        avails = pd.concat([avails, htmbs, atmbs])

    avails.columns=['fname', 'lname', 'b', 'pos', 'team']

    avails['name'] = avails.fname + ' ' + avails.lname

    avails = avails[['name', 'b', 'pos', 'team']]

    anames = []

    for i, x in avails.iterrows():
        if x['b'] == 'Jr.':
            anames.append(x['name'] + ' ' +  x['b'])  
        else:
            anames.append(x['name'])

    return all_sps + anames

def make_team_w_pos_ratchet(positions, method = 'points', df = None):
    """
    positions = list of strings, positions to be staffed,
                each position should be in the list the number
                of times it is in the roster
    
    returns a dataframe of a team formed with the efficient to
    best method. Starts with the most efficient players for 
    their price, then adds the best players, regardless of price, 
    while keeping the whole team's price under the cap
    """
    
    if df == None:
        df = prep_dataframe()
        
    df = df.sort_values('AvgPointsPerGame', ascending=False)
    
    best = make_best_team(df, positions = positions)
    
    swaps = 0
    
    while best.Salary.sum() > 50000:
        
        over = best.Salary.sum() - 50000
        
        swaps += 1
        print('swaps:', swaps)
        rmdf = df[~df['Name'].isin(best.Name)]
        
        loss = []
        alts = []
        diffs = []
        for i, r in best.iterrows():
            
            tdf = rmdf[rmdf['Position'] == r['Position']]
            tdf = tdf[tdf['Salary'] < r['Salary']]
            
            if len(tdf) > 0:
                ns, na = tdf.iloc[0][['Salary', 'AvgPointsPerGame']]
                os, oa = r[['Salary', 'AvgPointsPerGame']]
                alts.append(tdf.iloc[0])

    #             return ns, np, os, op

#                 ls = (os - ns) / (oa - na)
                if method == 'points':
                    ls = oa - na
                    
                if method == 'ratio':
                    ls = (oa - na) / (os - ns)
                
                diffs.append([os - ns])
                loss.append(ls)
            else:
                loss.append(10000.5)
                alts.append(None)
                diffs.append([os])
        
#         if any([x <= over] for x in)
#         loss = np.array(loss)
#         for i in range(len(diffs)):
#             if diffs[i] > over:
#                 loss[i] = max(loss)
        rep = np.array(loss).argmin()
        best.iloc[rep] = alts[rep]
    
    
            
#     print('Salary:', eff.Salary.sum())
#     print('Avg Points:', eff.AvgPointsPerGame.sum())
    print('Avg points:', best.AvgPointsPerGame.sum())
    print('Salary:', best.Salary.sum())
    
    return best


In [5]:
positions = ['C', '1B', '2B', 'SS', '3B', 'OF', 'OF', 'OF', 'P', 'P']

# df = pd.read_csv('DKSalaries.csv')
# df['PPK'] = df.AvgPointsPerGame / df.Salary

# # df = df[~df['Name'].isin(['Yoenis Cespedes', 'Robinson Cano'])]

# eff = make_efficient_team(df)

# best = make_best_team(df)

# eff['DEFICIT'] = best.AvgPointsPerGame - eff.AvgPointsPerGame
# best['DEFICIT'] = best.AvgPointsPerGame - eff.AvgPointsPerGame

# eff = eff.sort_values('DEFICIT', ascending=False).reset_index() 
# best = best.sort_values('DEFICIT', ascending=False).reset_index()

# for i, e in eff.iterrows():
    
#     ctotal = eff.Salary.sum()
#     ctminus = ctotal - e.Salary
    
#     if ctminus + best.Salary.loc[i] <= 50000:
#         eff.loc[i] = best.loc[i]
        


# eff.AvgPointsPerGame.sum()

# make_team_w_pos(df, positions)
# eff, best = make_team_flex(df, 6)

# eff['DEFICIT'] = best.AvgPointsPerGame - eff.AvgPointsPerGame
# eff

# # loss = 
# make_team
# make_team_w_pos_ratchet(positions)
# make_team_w_pos(positions)
df = make_team_w_pos_ratchet(positions, method = 'ratio')
export_team(df)
# df
# max([3,5,6])

AttributeError: 'NoneType' object has no attribute 'sort_values'

In [2]:
# var = []
# tdf = df.loc['P'].sort_values('AvgPointsPerGame', ascending=False)

# for i in range(len(tdf[:15]) - 1):
#     apg,salary = tdf.iloc[i][['AvgPointsPerGame', 'Salary']]
#     apg1,salary1 = tdf.iloc[i+1][['AvgPointsPerGame', 'Salary']]
#     div = salary - salary1 / 100
#     loss = (apg - apg1) / ((salary - salary1) / 100)

#     var.append(loss)
# var

# make_team_w_pos_ratchet(positions)
make_n_teams(1)

NameError: name 'positions' is not defined

In [59]:
# 13 * 15 * 1 * 1 * 16 * 7 * 15
# df.loc[df.index.unique()[5:]]
tdf
# df[((df['Position'] == 'OF') & (df['Salary'] <= 6000))]
# df.set_index('Roster Position', inplace=True)
# df.loc["P"]
pass

In [64]:
eff.Salary.sum()

49800

In [65]:
best.AvgPointsPerGame.sum()

143.88

In [367]:
# datetime.today() - timedelta(5)
avails.pos.unique()

array(['SS', '2B', '1B', 'LF', '3B', 'RF', 'CF', 'C', 'P', 'DH', '(R)'],
      dtype=object)

In [60]:
# for i, x in avails.iterrows():
#     if x['b'] == 'Jr.':
#         print(x['name'] + ' ' +  x['b'])  
#     else:
#         print(x['name'])

In [61]:
players = []
# rq = requests.get('https://rotogrinders.com/lineups/mlb?site=draftkings').content
souped = BeautifulSoup(rq, 'html.parser')
# ("span", {"class": "starting-lineups__team-name starting-lineups__team-name--home"})
divs = souped.findAll('div', {'class' : 'blk game'})
for div in divs:
    hm = div.find('div', {'class' : 'blk home-team'})
    aw = div.find('div', {'class' : 'blk away-team'})
    
    hp = hm.find('div', {'class' : 'pitcher players'}).find('a', {'class' : 'player-popup'}).text
    ap = aw.find('div', {'class' : 'pitcher players'}).find('a', {'class' : 'player-popup'}).text
    
    ht = hm.find('ul', {'class' : 'players unconfirmed'})#.findAll('li', {'class' : 'player'})
    at = aw.find('ul', {'class' : 'players unconfirmed'})#.findAll('li', {'class' : 'player'})
    
    ht = ht.findAll('li', {'class' : 'player'})
    ht = [x.find('span', {'class' : 'pname'}).text.strip() for x in ht]
    
    at = at.findAll('li', {'class' : 'player'})
    at = [x.find('span', {'class' : 'pname'}).text.strip() for x in at]
    
    players.extend([hp, ap])
    players.extend(at)
    players.extend(ht)
    
#     print(hp, ap)

# players

In [62]:
# rq = requests.get('https://legacy.baseballprospectus.com/sortable/index.php?cid=2508773').content
# souped = BeautifulSoup(rq, 'html.parser')
# ttdata = souped.find('table', {'id' : 'TTdata'})
# hdrs = [x.text for x in ttdata.find('tr').findAll('td')]
# data = []
# for r in ttdata.findAll('tr')[1:]:
#     data.append([x.text for x in r.findAll('td')])
    
# pf = pd.DataFrame(data, columns = hdrs)
# numheaders = list(pf)[3:]
# for header in numheaders:
#     pf[header] = pd.to_numeric(pf[header])
# pf = pf[pf['G'] > (pf['G'].mean() - pf["G"].std())]
# pf['WHIP'] = (pf['H'] + pf['BB'] + pf['HBP']) / pf['IP']
# # pf.dtypes
# pf['MULT'] = pf['WHIP'] / pf['WHIP'].mean()
# pf.sort_values('MULT', ascending=False)
# pf
# pf['G'].describe()

# make

In [103]:
df.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame,OpposingTeam
0,SP,Max Scherzer (11036841),Max Scherzer,11036841,P,13900,ATL@WAS 08/07/2018 07:05PM ET,WAS,26.57,ATL
1,SP,Chris Sale (11037695),Chris Sale,11037695,P,13500,BOS@TOR 08/07/2018 07:07PM ET,BOS,28.68,TOR
2,SP,James Paxton (11037696),James Paxton,11037696,P,12900,SEA@TEX 08/07/2018 08:05PM ET,SEA,22.67,TEX
3,SP,Aaron Nola (11037698),Aaron Nola,11037698,P,12400,PHI@ARI 08/07/2018 09:40PM ET,PHI,21.78,ARI
4,SP,Jacob deGrom (11037697),Jacob deGrom,11037697,P,12400,CIN@NYM 08/07/2018 07:10PM ET,NYM,24.87,CIN


In [100]:
df.head()

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
0,SP,Max Scherzer (11036841),Max Scherzer,11036841,P,13900,ATL@WAS 08/07/2018 07:05PM ET,WAS,26.57
1,SP,Chris Sale (11037695),Chris Sale,11037695,P,13500,BOS@TOR 08/07/2018 07:07PM ET,BOS,28.68
2,SP,James Paxton (11037696),James Paxton,11037696,P,12900,SEA@TEX 08/07/2018 08:05PM ET,SEA,22.67
3,SP,Aaron Nola (11037698),Aaron Nola,11037698,P,12400,PHI@ARI 08/07/2018 09:40PM ET,PHI,21.78
4,SP,Jacob deGrom (11037697),Jacob deGrom,11037697,P,12400,CIN@NYM 08/07/2018 07:10PM ET,NYM,24.87


In [124]:
get_projected_players()
# hm.find('ul').findAll('li', {'class' : 'player'})

['Kyle Hendricks',
 'Jeremy Hellickson',
 'Adam Eaton',
 'Trea Turner',
 'Anthony Rendon',
 'Bryce Harper',
 'Juan Soto',
 'Ryan Zimmerman',
 'Daniel Murphy',
 'Spencer Kieboom',
 'Jeremy Hellickson',
 'Anthony Rizzo',
 'Javier Baez',
 'Ben Zobrist',
 'Jason Heyward',
 'David Bote',
 'Kyle Schwarber',
 'Willson Contreras',
 'Kyle Hendricks',
 'Addison Russell',
 'Masahiro Tanaka',
 'Mike Minor',
 'Shin-Soo Choo',
 'Rougned Odor',
 'Elvis Andrus',
 'Adrian Beltre',
 'Jurickson Profar',
 'Ronald Guzman',
 'Willie Calhoun',
 'Isiah Kiner-Falefa',
 'Drew Robinson',
 'Aaron Hicks',
 'Giancarlo Stanton',
 'Gleyber Torres',
 'Didi Gregorius',
 'Miguel Andujar',
 'Luke Voit',
 'Brett Gardner',
 'Austin Romine',
 'Shane Robinson',
 'Dylan Bundy',
 'Nathan Eovaldi',
 'Mookie Betts',
 'Andrew Benintendi',
 'Mitch Moreland',
 'J.D. Martinez',
 'Xander Bogaerts',
 'Rafael Devers',
 'Brock Holt',
 'Daniel Butler',
 'Jackie Bradley',
 'Jonathan Villar',
 'Tim Beckham',
 'Adam Jones',
 'Mark Trumbo',


In [63]:
# df = prep_dataframe()

# pd.merge([df, df[df['Position'] == 'SP']['N']]how='left')


In [182]:
rq = requests.get('https://www.teamrankings.com/mlb/stat/runs-per-game').content
souped = BeautifulSoup(rq, 'html.parser')
table = souped.find('table', {'class' : 'tr-table datatable scrollable'})
rows = table.findAll('tr')
header = [x.text for x in rows[0].findAll('th')]
content = []
for row in rows[1:]:
    content.append([x.text for x in row.findAll('td')])
    
tmdf = pd.DataFrame(content, columns = header)
tmdf ['Team'] = [team_dict[x] for x in tmdf.Team]
tmdf['2018'] = pd.to_numeric(tmdf['2018'])
tmdf['MOD'] = tmdf['2018'].mean() / tmdf['2018']
tmdf[['Team', 'MOD']]
# tmdf.iloc[:, 1:3]
# souped

,Team,MOD
0,BOS,0.826407
1,NYY,0.861573
2,CLE,0.871689
3,HOU,0.898051
4,CHC,0.898051
5,TEX,0.929923
6,ATL,0.935784
7,LAD,0.937754
8,OAK,0.943715
9,COL,0.949751


In [198]:
ddf = prep_dataframe()
ddf
# ddf.MULT_y.unique()
# ddf[['OpposingTeam', 'MULT_y']]

# ddf[ddf['OpposingTeam'] == 'NYY']

,AvgPointsPerGame,Game Info,ID,MULT_y,NAME,Name,Name + ID,OppPitcher,OpposingTeam,PPK,Position,Roster Position,Salary,Team,TeamAbbrev,index,Adjusted
0,26.16,SEA@HOU 08/09/2018 08:10PM ET,11047795,1.060556,NaN,Gerrit Cole,Gerrit Cole (11047795),Mike Leake,SEA,2.076190,SP,P,12600,SEA,HOU,67,27.744133
1,21.76,BAL@TB 08/09/2018 07:10PM ET,11047418,1.139216,NaN,Blake Snell,Blake Snell (11047418),Dylan Bundy,BAL,1.892174,SP,P,11500,BAL,TB,29,24.789333
2,18.65,TEX@NYY 08/09/2018 07:05PM ET,11047798,0.929923,NaN,Masahiro Tanaka,Masahiro Tanaka (11047798),Mike Minor,TEX,1.810680,SP,P,10300,TEX,NYY,38,17.343072
3,17.80,LAD@COL 08/09/2018 08:40PM ET,11047804,0.937754,NaN,Jon Gray,Jon Gray (11047804),Kenta Maeda,LAD,2.022727,SP,P,8800,LAD,COL,24,16.692028
4,15.52,LAD@COL 08/09/2018 08:40PM ET,11047812,0.949751,NaN,Kenta Maeda,Kenta Maeda (11047812),Jon Gray,COL,1.869880,SP,P,8300,COL,LAD,17,14.740139
5,16.38,BOS@TOR 08/09/2018 07:07PM ET,11047816,0.978974,NaN,Nathan Eovaldi,Nathan Eovaldi (11047816),Marco Estrada,TOR,2.127273,SP,P,7700,TOR,BOS,74,16.035600
6,16.58,BAL@TB 08/09/2018 07:10PM ET,11047817,1.058036,NaN,Dylan Bundy,Dylan Bundy (11047817),Blake Snell,TB,2.181579,SP,P,7600,TB,BAL,5,17.542244
7,13.75,TEX@NYY 08/09/2018 07:05PM ET,11047824,0.861573,NaN,Mike Minor,Mike Minor (11047824),Masahiro Tanaka,NYY,1.936620,SP,P,7100,NYY,TEX,8,11.846631
8,12.83,PIT@SF 08/09/2018 10:15PM ET,11047836,0.981131,NaN,Derek Holland,Derek Holland (11047836),Clay Holmes,PIT,2.004688,SP,P,6400,PIT,SF,338,12.587907
9,12.10,SEA@HOU 08/09/2018 08:10PM ET,11047851,0.898051,NaN,Mike Leake,Mike Leake (11047851),Gerrit Cole,HOU,2.122807,SP,P,5700,HOU,SEA,2,10.866418


In [4]:
df.TeamAbbrev.unique()

AttributeError: 'DataFrame' object has no attribute 'TeamAbbrev'

In [15]:
teams = make_n_teams(1, positions)
export_team(teams)
teams

Extracting csv
Filtering by projected players
Expanding dataframe
Getting pitcher and team score information
swaps: 5
swaps: 6
Avg points (adj): 123.09943840652352
Avg points: 89.12
Salary: 50000
Salary: 47300
Avg Points: 91.16
Avg w/o cap: 91.84
Sal w/o cap: 51400
ratchet


[                 AvgPointsPerGame                      Game Info        ID  \
 Roster Position                                                              
 C                            5.07  OAK@MIN 08/23/2018 08:10PM ET  11121044   
 1B                           7.22  OAK@MIN 08/23/2018 08:10PM ET  11120930   
 2B                           7.92  OAK@MIN 08/23/2018 08:10PM ET  11120916   
 SS                           7.56  OAK@MIN 08/23/2018 08:10PM ET  11120921   
 3B                           8.12  OAK@MIN 08/23/2018 08:10PM ET  11120922   
 OF                           9.66  OAK@MIN 08/23/2018 08:10PM ET  11120913   
 OF                           5.72    KC@TB 08/23/2018 07:10PM ET  11121007   
 OF                           4.46    KC@TB 08/23/2018 07:10PM ET  11121057   
 P                           15.48  ATL@MIA 08/23/2018 07:10PM ET  11120909   
 P                           17.91  OAK@MIN 08/23/2018 08:10PM ET  11120906   
 
                      MULT           NAME         

,P,P,C,1B,2B,3B,SS,OF,OF,OF
0,11057169,11057171,11057596,11057549,11057462,11057193,11057472,11057236,11057563,11057654
0,11057170,11057172,11057303,11057359,11057448,11057538,11057192,11057487,11057507,11057458
0,11057791,11057183,11057287,11057228,11057242,11057349,11057328,11057319,11057237,11057340


In [249]:
get_pitcher_adjustments().describe()

,MULT
count,603.000000
mean,1.000000
std,0.293691
min,0.236190
25%,0.832518
50%,0.945883
75%,1.087365
max,3.320969


In [88]:
url = "https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=p&\
lastndays=5&since=2018-05-01&dates=fromandto&fromandto=2018-08-%s.2018-08-%s&level=mlb&franch=ANY&stat_value=0"
# https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=b&dates=fromandto\
# &lastndays=7&since=2018-08-01&fromandto=2018-08-%s.2018-08-%s&level=mlb&franch=ANY&stat_value=0"
url2 = "https://www.baseball-reference.com/leagues/daily.fcgi?request=1&type=p&\
user_team=&bust_cache=&lastndays=7&dates=since&since=2018-03-01&fromandto=2018-08-01.2018-08-31&\
level=mlb&franch=ANY&stat=&stat_value=0"
stats = []

for i in range(1, 2):
# #     if i < 10:
# #         k = '0' + str(i)
# #     else:
# #         k = str(i)
# #     dt = '2018-08-%s' % (k)
#     uurl = url % (k , k)
#     print(uurl)
    ct = requests.get(url2).content
    souped = BeautifulSoup(ct, 'html.parser') 
    souped.find('thead').text.split('\n')
    head = souped.find('thead')
    head = [x.text for x in head.findAll('th')][1:] + ['DT']
    table = souped.find('tbody')
    rows = table.findAll('tr')
    print(souped.find('div', {'id':"desc_container"}).text)
    for row in rows:
        stats.append([x.text for x in row.findAll('td')] + [dt])
    
df = pd.DataFrame(stats, columns=head)
df


Current search:
Major league pitching logs since Mar  1
    



,Name,,Age,#days,Lev,Tm,G,GS,W,L,...,StL,StS,GB/FB,LD,PU,WHIP,BAbip,SO9,SO/W,DT
0,Jason Adam,gl,26,7,MLB-AL,Kansas City,31,0,,3,...,12%,13%,29%,22%,19%,1.392,.269,10.3,2.47,2018-08-22
1,Austin L. Adams,gl,27,118,MLB-NL,Washington,2,0,,,...,25%,4%,50%,25%,0%,4.000,.250,0.0,0.00,2018-08-22
2,Chance Adams,gl,23,2,MLB-AL,New York,2,1,,1,...,15%,4%,42%,33%,0%,1.650,.190,2.7,0.50,2018-08-22
3,Matt Albers,gl,35,0,MLB-NL,Milwaukee,30,0,3,3,...,14%,11%,45%,20%,11%,1.467,.310,8.2,3.11,2018-08-22
4,Sandy Alcantara,gl,22,56,MLB-NL,Miami,1,1,1,,...,14%,6%,40%,27%,13%,1.600,.214,3.6,0.40,2018-08-22
5,Victor Alcantara,gl,25,0,MLB-AL,Detroit,16,0,,,...,14%,10%,47%,25%,12%,1.034,.232,6.5,3.50,2018-08-22
6,Scott Alexander,gl,28,2,MLB-NL,Los Angeles,58,2,2,1,...,16%,12%,72%,18%,2%,1.291,.297,7.4,1.88,2018-08-22
7,Kolby Allard,gl,20,7,MLB-NL,Atlanta,3,1,1,1,...,13%,5%,36%,26%,5%,2.875,.457,3.4,0.75,2018-08-22
8,Cody Allen,gl,29,0,MLB-AL,Cleveland,55,0,4,5,...,16%,13%,30%,24%,9%,1.296,.279,10.5,2.74,2018-08-22
9,Miguel Almonte,gl,25,50,MLB-AL,Los Angeles,1,0,,,...,18%,18%,25%,25%,0%,2.000,.333,9.0,,2018-08-22


In [89]:
df.to_csv('./pitcher_season_stats.csv')

In [64]:
# url3 = "https://www.baseball-reference.com/leagues/MLB-schedule.shtml"

# rq = requests.get(url3).content
# souped = BeautifulSoup(rq, 'html.parser')
ct = souped.find('div', {'class' : 'section_content'})
divs = ct.findAll('div')
gdf = pd.DataFrame()
k = 0
for div in divs[:146]:
    k+=1
    print(k)
    dt = div.find('h3').text
#     print(dt)
    dt = datetime.strptime(dt, '%A, %B %d, %Y').date()
#     print(dt)
    gms = [x.text for x in div.findAll('p')]
    gms = [x.split('\n') for x in gms]
    gms = [ [x[1].strip(), x[4].strip()] for x in gms if len(x) > 1]
    tdf = pd.DataFrame(gms)
    tdf['date'] = dt
    gdf = pd.concat([gdf, tdf])
#     print(gms)
    
gdf.columns = ['away', 'home', 'date']
# gdf

AttributeError: 'NoneType' object has no attribute 'findAll'

In [65]:
# gdf.to_csv('2018_game_schedule_according_to_br.csv')

url = "http://m.angels.mlb.com/%s/roster/40-man/"

tms = ['ana', 'bos']

players = pd.DataFrame()

for tm in list(team_dict.values()) + ['ana']:
    
    url2 = url % (tm.lower().strip())
    print(url2)

    rq = requests.get(url2)
    print(rq.status_code)
    rq = rq.content
    souped = BeautifulSoup(rq, 'html.parser')
    module = souped.findAll('section', {'class' : 'module'})
    tmdata = []
    count = 0
    for mod in module:
        for row in mod.findAll('tr'):
            try:
    #             print('found')
                nm = row.find('td', {'class' : 'dg-name_display_first_last'}).text.strip()
                hands = row.find('td', {'class' : 'dg-bats_throws'}).text
                if '(' in nm:
                    nm = nm.split('(')[0].strip()
                bats, throws = [x.strip() for x in hands.split('/')]
                tmdata.append([nm, bats, throws])
        #         print(nm, bats, throws)

            except:
    #             print('not found')
                pass
            count += 1
    print(count)
    tdf = pd.DataFrame(tmdata)
    players = pd.concat([players, tdf])
    
# players

http://m.angels.mlb.com/bos/roster/40-man/
200
48
http://m.angels.mlb.com/nyy/roster/40-man/
200
47
http://m.angels.mlb.com/cle/roster/40-man/
200
52
http://m.angels.mlb.com/hou/roster/40-man/
200
44
http://m.angels.mlb.com/chc/roster/40-man/
200
48
http://m.angels.mlb.com/tex/roster/40-man/
200
50
http://m.angels.mlb.com/atl/roster/40-man/
200
53
http://m.angels.mlb.com/lad/roster/40-man/
200
50
http://m.angels.mlb.com/oak/roster/40-man/
200
50
http://m.angels.mlb.com/col/roster/40-man/
200
47
http://m.angels.mlb.com/wsh/roster/40-man/
200
46
http://m.angels.mlb.com/laa/roster/40-man/
200
54
http://m.angels.mlb.com/cin/roster/40-man/
200
47
http://m.angels.mlb.com/tor/roster/40-man/
200
47
http://m.angels.mlb.com/pit/roster/40-man/
200
46
http://m.angels.mlb.com/ari/roster/40-man/
200
48
http://m.angels.mlb.com/stl/roster/40-man/
200
48
http://m.angels.mlb.com/min/roster/40-man/
200
48
http://m.angels.mlb.com/mil/roster/40-man/
200
48
http://m.angels.mlb.com/phi/roster/40-man/
200
44


In [40]:
# players[0].unique()
# team_dict.values()
players.columns = ['name', 'bats', 'throws']
players.to_csv('team_rosters.csv')
# rq.status_code

In [78]:
df = pd.read_csv('./game_logs_raw_batters.csv')
df.dropna(inplace=True)
list(df)
# pdict = {
#     '1B', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'HBP', 'SB'
# }
mults = np.array([3, 5, 8, 10, 2, 2, 2, 2, 2, 5])
cols = ['Name', '1B', '2B', '3B', 'HR', 'RBI', 'R', 'BB', 'IBB', 'HBP', 'SB']
# df['1B'] = df['H'] - 

df['1B'] = df['H'] - df[['2B', '3B', 'HR']].sum(axis=1)
df[cols]
df['VAL'] = (df[cols[1:]] * mults).sum(axis=1)
# df[cols + ['VAL']]
df

,Unnamed: 0,Name,Unnamed: 2,Age,#days,Lev,Tm,G,PA,AB,...,SF,GDP,SB,CS,BA,OBP,SLG,OPS,1B,VAL
0,0,Jose Abreu,gl,31.0,21.0,MLB-AL,Chicago,1.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.333,0.500,0.667,1.167,0.0,11.0
1,1,Willy Adames,gl,22.0,21.0,MLB-AL,Tampa Bay,1.0,4.0,4.0,...,0.0,1.0,0.0,0.0,0.500,0.500,1.250,1.750,1.0,21.0
2,2,Matt Adams,gl,29.0,21.0,MLB-NL,Washington,1.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,2.0
3,3,Jim Adduci,gl,33.0,21.0,MLB-AL,Detroit,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,1.000,1.000,2.500,3.500,1.0,19.0
4,4,Ehire Adrianza,gl,28.0,21.0,MLB-AL,Minnesota,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
5,5,Jesus Aguilar,gl,28.0,21.0,MLB-NL,Milwaukee,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.333,0.600,0.667,1.267,0.0,11.0
6,6,Albert Almora,gl,24.0,21.0,MLB-NL,Chicago,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
7,7,Yonder Alonso,gl,31.0,21.0,MLB-AL,Cleveland,1.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
8,8,Chase Anderson,gl,30.0,21.0,MLB-NL,Milwaukee,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.0,0.0
9,9,Tim Anderson,gl,25.0,21.0,MLB-AL,Chicago,1.0,4.0,4.0,...,0.0,0.0,0.0,1.0,0.500,0.500,0.750,1.250,1.0,8.0


,Name,,Age,#days,Lev,Tm,G,GS,W,L,...,StS,GB/FB,LD,PU,WHIP,BAbip,SO9,SO/W,DT,ftm
0,Jason Adam,gl,26,7,MLB-AL,Kansas City,31,0,,3,...,13%,29%,22%,19%,1.392,.269,10.3,2.47,2018-08-22,Kansas City Royals
1,Austin L. Adams,gl,27,118,MLB-NL,Washington,2,0,,,...,4%,50%,25%,0%,4.000,.250,0.0,0.00,2018-08-22,Washington Nationals
2,Chance Adams,gl,23,2,MLB-AL,New York,2,1,,1,...,4%,42%,33%,0%,1.650,.190,2.7,0.50,2018-08-22,New York Yankees
3,Matt Albers,gl,35,0,MLB-NL,Milwaukee,30,0,3,3,...,11%,45%,20%,11%,1.467,.310,8.2,3.11,2018-08-22,Milwaukee Brewers
4,Sandy Alcantara,gl,22,56,MLB-NL,Miami,1,1,1,,...,6%,40%,27%,13%,1.600,.214,3.6,0.40,2018-08-22,Miami Marlins


In [160]:
df[df['Tm'] == 'Washington,Chicago']

,Name,,Age,#days,Lev,Tm,G,GS,W,L,...,StL,StS,GB/FB,LD,PU,WHIP,BAbip,SO9,SO/W,DT
377,Brandon Kintzler,gl,33,2,MLB-NL,"Washington,Chicago",55,0,1,3,...,19%,7%,49%,29%,5%,1.309,.299,6.3,1.94,2018-08-22


In [172]:
gdf

,away,home,date
0,Colorado Rockies,Arizona D'Backs,2018-03-29
1,Philadelphia Phillies,Atlanta Braves,2018-03-29
2,Minnesota Twins,Baltimore Orioles,2018-03-29
3,Chicago White Sox,Kansas City Royals,2018-03-29
4,San Francisco Giants,Los Angeles Dodgers,2018-03-29
5,Chicago Cubs,Miami Marlins,2018-03-29
6,St. Louis Cardinals,New York Mets,2018-03-29
7,Los Angeles Angels,Oakland Athletics,2018-03-29
8,Milwaukee Brewers,San Diego Padres,2018-03-29
9,Cleveland Indians,Seattle Mariners,2018-03-29


In [42]:
gdf = pd.read_csv('./2018_game_schedule_according_to_br.csv')
match_dict = {}
for i, x in gdf.iterrows():
    dt_dict = match_dict.get(x['date'], {})
    dt_dict[x['away']] = {'home' : 0, 'opponent' : x['home']}
    dt_dict[x['home']] = {'home' : 1, 'opponent' : x['away']}
    match_dict[x['date']] = dt_dict


df = pd.read_csv('./game_logs_raw_batters.csv')



df.dropna(inplace=True)
list(df)
# pdict = {
#     '1B', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'HBP', 'SB'
# }
mults = np.array([3, 5, 8, 10, 2, 2, 2, 2, 2, 5])
cols = ['Name', '1B', '2B', '3B', 'HR', 'RBI', 'R', 'BB', 'IBB', 'HBP', 'SB']
# df['1B'] = df['H'] - 

df['1B'] = df['H'] - df[['2B', '3B', 'HR']].sum(axis=1)
df[cols]
df['VAL'] = (df[cols[1:]] * mults).sum(axis=1)


al_dict = {}
nl_dict = {}

al = df[df['Lev'] == 'MLB-AL']['Tm'].unique()
nl = df[df['Lev'] == 'MLB-NL']['Tm'].unique()

all_tms = gdf.away.unique()

for tm in al:
    for t2 in all_tms:
        if tm in t2 and t2 not in ['Chicago Cubs', 'New York Mets', 'Los Angeles Dodgers']:
            al_dict[tm] = t2
            
for tm in nl:
    for t2 in all_tms:
        if tm in t2 and t2 not in ['Chicago White Sox', 'New York Yankees', 'Los Angeles Angels']:
            nl_dict[tm] = t2
            

# set(nl).intersection(set(al))
al = df[df['Lev'] == 'MLB-AL']
nl = df[df['Lev'] == 'MLB-NL']

df['ftm'] = 'None'
ftm = []
for i, x in df.iterrows():
    if x['Lev'] == 'MLB-AL':
        ftm.append( al_dict.get(x['Tm'], 'None'))
    else:
        ftm.append( nl_dict.get(x['Tm'], 'None'))

df['ftm'] = ftm

df.dropna(inplace=True)
home = []
opponent = []
for i, x in df.iterrows():
#     print(x['DT'])
#     dt = datetime.strptime(x['DT'], "%Y-%m-%d").date()
    
    dt = x['DT']
#     print(match_dict[dt])
    home.append(match_dict[dt][x['ftm']]['home'])
    opponent.append(match_dict[dt][x['ftm']]['opponent'])

df['home'] = home
df['opponent'] = opponent

trim_heads = [
 'Name',
 'Unnamed: 2',
 'DT',
 'VAL',
 'ftm',
 'home',
 'opponent']

df = df[trim_heads]

df2 = pd.read_csv('./batter_season_stats.csv')

df = pd.merge(df, df2, how='left', on='Name')

df3 = pd.read_csv('./game_logs_raw_pitchers.csv')



ftm = []
for i, x in df3.iterrows():
    if x['Lev'] == 'MLB-AL':
        ftm.append( al_dict.get(x['Tm'], 'None'))
    else:
        ftm.append( nl_dict.get(x['Tm'], 'None'))

df3['ftm'] = ftm

df3 = df3[['Name', 'Tm', 'DT', 'ftm']]

op_dict = {}
for i, x in df3.iterrows():
    ddict = op_dict.get(x['DT'], {})
    ddict[x['ftm']] = x['Name']
    op_dict[x['DT']] = ddict
    
op = []
for i, x in df.iterrows():
#     print(x['DT_y'], type(x['DT_y']))
    op.append(op_dict[x['DT_x']][x['opponent']])

df['op'] = op

df4 = pd.read_csv('./pitcher_season_stats.csv')

df = pd.merge(df, df4, how='left', left_on='op', right_on='Name')

df['Name'] = df['Name_x']

# df = df.drop(['Name_y', 'Name_x'], axis=1)

df5 = pd.read_csv('./team_rosters.csv')
# df5.columns = ['Unnamed: 0', 'Name', 'bats', 'throws']

df = pd.merge(df, df5[['name', 'throws']], how='left', left_on='op', right_on='name')
df = pd.merge(df, df5[['name', 'bats']], how='left', left_on='Name_x', right_on='name')

df['opp_hand'] = df['throws'] != df['bats']

df.head(25)
# df3.head()

,Name_x,Unnamed: 2_x,DT_x,VAL,ftm,home,opponent,Unnamed: 0_x,Unnamed: 2_y,Age_x,...,BAbip,SO9,SO/W,DT,Name,name_x,throws,name_y,bats,opp_hand
0,Jose Abreu,gl,2018-08-01,11.0,Chicago White Sox,1,Kansas City Royals,0,gl,31.0,...,0.355,8.3,2.00,2018-08-22,Jose Abreu,Glenn Sparkman,R,Jose Abreu,R,False
1,Willy Adames,gl,2018-08-01,21.0,Tampa Bay Rays,1,Los Angeles Angels,2,gl,22.0,...,0.256,7.6,2.06,2018-08-22,Willy Adames,Nick Tropeano,R,Willy Adames,R,False
2,Willy Adames,gl,2018-08-01,21.0,Tampa Bay Rays,1,Los Angeles Angels,2,gl,22.0,...,0.256,7.6,2.06,2018-08-22,Willy Adames,Nick Tropeano,R,Willy Adames,R,False
3,Matt Adams,gl,2018-08-01,2.0,Washington Nationals,1,New York Mets,4,gl,29.0,...,0.343,9.4,5.05,2018-08-22,Matt Adams,Noah Syndergaard,R,Matt Adams,L,True
4,Jim Adduci,gl,2018-08-01,19.0,Detroit Tigers,1,Cincinnati Reds,5,gl,33.0,...,0.290,6.5,1.90,2018-08-22,Jim Adduci,Sal Romano,R,Jim Adduci,L,True
5,Ehire Adrianza,gl,2018-08-01,0.0,Minnesota Twins,1,Cleveland Indians,6,gl,28.0,...,0.289,12.7,4.20,2018-08-22,Ehire Adrianza,Brad Hand,L,Ehire Adrianza,S,True
6,Jesus Aguilar,gl,2018-08-01,11.0,Milwaukee Brewers,0,Los Angeles Dodgers,7,gl,28.0,...,0.227,10.1,4.50,2018-08-22,Jesus Aguilar,Kenley Jansen,R,Jesus Aguilar,R,False
7,Albert Almora,gl,2018-08-01,0.0,Chicago Cubs,0,Pittsburgh Pirates,19,gl,24.0,...,0.328,10.8,4.13,2018-08-22,Albert Almora,Richard Rodriguez,R,NaN,NaN,True
8,Yonder Alonso,gl,2018-08-01,0.0,Cleveland Indians,0,Minnesota Twins,20,gl,31.0,...,0.303,10.1,2.85,2018-08-22,Yonder Alonso,Fernando Rodney,R,Yonder Alonso,L,True
9,Chase Anderson,gl,2018-08-01,0.0,Milwaukee Brewers,0,Los Angeles Dodgers,28,gl,30.0,...,0.227,10.1,4.50,2018-08-22,Chase Anderson,Kenley Jansen,R,Chase Anderson,R,False


In [55]:
lcol = 'VAL'

tcols = [
 'ftm',
 'home',
 'opponent',
 'Age_x',
 '#days_x',
#  'Lev_x',
#  'Tm_x',
#  'G_x',
#  'PA',
#  'AB_x',
#  'R_x',
#  'H_x',
#  '2B_x',
#  '3B_x',
#  'HR_x',
#  'RBI',
#  'BB_x',
#  'IBB_x',
#  'SO_x',
#  'HBP_x',
#  'SH',
#  'SF_x',
#  'GDP_x',
#  'SB_x',
#  'CS_x',
 'BA',
 'OBP',
 'SLG',
 'OPS',
#  'DT_y',
#  'op',
#  'Unnamed: 0_y',
#  'Name_y',
#  'Unnamed: 2',
#  'Age_y',
#  '#days_y',
#  'Lev_y',
#  'Tm_y',
#  'G_y',
#  'GS',
#  'W',
#  'L',
#  'SV',
#  'IP',
#  'H_y',
#  'R_y',
#  'ER',
#  'BB_y',
#  'SO_y',
#  'HR_y',
#  'HBP_y',
 'ERA',
#  'AB_y',
#  '2B_y',
#  '3B_y',
#  'IBB_y',
#  'GDP_y',
#  'SF_y',
#  'SB_y',
#  'CS_y',
 'PO',
 'BF',
 'Pit',
 'Str',
 'StL',
 'StS',
 'GB/FB',
 'LD',
 'PU',
 'WHIP',
 'BAbip',
 'SO9',
 'SO/W',
#  'DT',
#  'Name',
#  'name_x',
 'throws',
#  'name_y',
 'bats',
 'opp_hand']

df_bat_train = df[~df['Name_x'].isin(df.name_x.unique())]
df_bat_train[tcols + ['VAL']].to_csv('batter_training_set.csv')
# df[tcols + ['VAL']].to_csv('training_set.csv')

In [241]:
list(df4)

['Unnamed: 0',
 'Name',
 'Unnamed: 2',
 'Age',
 '#days',
 'Lev',
 'Tm',
 'G',
 'GS',
 'W',
 'L',
 'SV',
 'IP',
 'H',
 'R',
 'ER',
 'BB',
 'SO',
 'HR',
 'HBP',
 'ERA',
 'AB',
 '2B',
 '3B',
 'IBB',
 'GDP',
 'SF',
 'SB',
 'CS',
 'PO',
 'BF',
 'Pit',
 'Str',
 'StL',
 'StS',
 'GB/FB',
 'LD',
 'PU',
 'WHIP',
 'BAbip',
 'SO9',
 'SO/W',
 'DT']

In [196]:
df.head()

,Unnamed: 0,Name,Unnamed: 2,Age,#days,Lev,Tm,G,PA,AB,...,SF,GDP,SB,CS,BA,OBP,SLG,OPS,DT,ftm
0,0,Jose Abreu,gl,31.0,21.0,MLB-AL,Chicago,1.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.333,0.5,0.667,1.167,2018-08-01,Chicago White Sox
1,1,Willy Adames,gl,22.0,21.0,MLB-AL,Tampa Bay,1.0,4.0,4.0,...,0.0,1.0,0.0,0.0,0.500,0.5,1.250,1.750,2018-08-01,Tampa Bay Rays
2,2,Matt Adams,gl,29.0,21.0,MLB-NL,Washington,1.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.000,2018-08-01,Washington Nationals
3,3,Jim Adduci,gl,33.0,21.0,MLB-AL,Detroit,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,1.000,1.0,2.500,3.500,2018-08-01,Detroit Tigers
4,4,Ehire Adrianza,gl,28.0,21.0,MLB-AL,Minnesota,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.000,0.0,0.000,0.000,2018-08-01,Minnesota Twins


In [210]:
df

,Unnamed: 0,Name,Unnamed: 2,Age,#days,Lev,Tm,G,PA,AB,...,SB,CS,BA,OBP,SLG,OPS,DT,ftm,home,opponent
0,0,Jose Abreu,gl,31.0,21.0,MLB-AL,Chicago,1.0,4.0,3.0,...,0.0,0.0,0.333,0.500,0.667,1.167,2018-08-01,Chicago White Sox,1,Kansas City Royals
1,1,Willy Adames,gl,22.0,21.0,MLB-AL,Tampa Bay,1.0,4.0,4.0,...,0.0,0.0,0.500,0.500,1.250,1.750,2018-08-01,Tampa Bay Rays,1,Los Angeles Angels
2,2,Matt Adams,gl,29.0,21.0,MLB-NL,Washington,1.0,4.0,4.0,...,0.0,0.0,0.000,0.000,0.000,0.000,2018-08-01,Washington Nationals,1,New York Mets
3,3,Jim Adduci,gl,33.0,21.0,MLB-AL,Detroit,1.0,2.0,2.0,...,0.0,0.0,1.000,1.000,2.500,3.500,2018-08-01,Detroit Tigers,1,Cincinnati Reds
4,4,Ehire Adrianza,gl,28.0,21.0,MLB-AL,Minnesota,1.0,2.0,2.0,...,0.0,0.0,0.000,0.000,0.000,0.000,2018-08-01,Minnesota Twins,1,Cleveland Indians
5,5,Jesus Aguilar,gl,28.0,21.0,MLB-NL,Milwaukee,1.0,5.0,3.0,...,0.0,0.0,0.333,0.600,0.667,1.267,2018-08-01,Milwaukee Brewers,0,Los Angeles Dodgers
6,6,Albert Almora,gl,24.0,21.0,MLB-NL,Chicago,1.0,1.0,1.0,...,0.0,0.0,0.000,0.000,0.000,0.000,2018-08-01,Chicago Cubs,0,Pittsburgh Pirates
7,7,Yonder Alonso,gl,31.0,21.0,MLB-AL,Cleveland,1.0,4.0,4.0,...,0.0,0.0,0.000,0.000,0.000,0.000,2018-08-01,Cleveland Indians,0,Minnesota Twins
8,8,Chase Anderson,gl,30.0,21.0,MLB-NL,Milwaukee,1.0,2.0,2.0,...,0.0,0.0,0.000,0.000,0.000,0.000,2018-08-01,Milwaukee Brewers,0,Los Angeles Dodgers
9,9,Tim Anderson,gl,25.0,21.0,MLB-AL,Chicago,1.0,4.0,4.0,...,0.0,1.0,0.500,0.500,0.750,1.250,2018-08-01,Chicago White Sox,1,Kansas City Royals


In [221]:
df3

,Name,Tm,DT
0,Matt Albers,Milwaukee,2018-08-01
1,Scott Alexander,Los Angeles,2018-08-01
2,Yency Almonte,Colorado,2018-08-01
3,Jose Alvarado,Tampa Bay,2018-08-01
4,Jose Alvarez,Los Angeles,2018-08-01
5,Chase Anderson,Milwaukee,2018-08-01
6,Danny Barnes,Toronto,2018-08-01
7,Steven Brault,Pittsburgh,2018-08-01
8,Ryan Buchter,Oakland,2018-08-01
9,Corbin Burnes,Milwaukee,2018-08-01


In [95]:
# list(df)
export_team([randomized_best_team(prep_dataframe(), positions)])

Extracting csv
Filtering by projected players
Expanding dataframe
Getting pitcher and team score information
0
0
3
3
6
3
6
0
14
8
16
2
31
15
339
308
2105
1766
3625
1520
6554
2929
Counter({'loss': 8932, 'sal': 2622, 'win': 11})


,P,P,C,1B,2B,3B,SS,OF,OF,OF
0,11236256,11236878,11236456,11236433,11236359,11236572,11236638,11236394,11236347,11236340
